## Download all the dependencies

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.19.0                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


### 1. Data Collection
As mentioned previously, it was used data from 2 different sources. The first database is about Paris + nearby areas hotels and contains information related to postal code, name of the hotel, address. It contains 40 observations about 40 Paris + nearby areas hotels. For a better analysis it was selected data about hotels from different areas and with different facilities. The second source used is Foursquare location data in order to explore and target recommended locations across different venues. Everything was arranged into one single pandas dataframe that can be seen below. The data was collected by me and it was processed before to import it in this notebook.

In [11]:
# Read the Paris hotels data (Source: www.accorhotels.com)
hotels = pd.read_csv('https://raw.githubusercontent.com/mysee1984/Coursera_Capstone/master/Hotels.csv',encoding = "ISO-8859-1")
hotels

,PostalCode,Hotel_Name,Address
0,75008,Mercure Paris Opera Garnier Hotel,4 rue de l 'Isly
1,75009,Scribe Paris Opéra by Sofitel,1 rue Scribe
2,75017,Mercure Paris St Lazare Monceau hotel,99 bis Rue de Rome
3,75002,Hôtel Stendhal Place Vendôme Paris - MGallery ...,22 rue Danielle Casanova
4,75018,Mercure Paris Montmartre Sacré-Coeur Hotel,3 rue Caulaincourt
5,75010,Hôtel L'Échiquier Opéra Paris - MGallery by So...,38 Rue De L Echiquier
6,75015,ibis Paris Tour Montparnasse 15th,22 Avenue du Maine
7,75014,Mercure Paris Gare Montparnasse Hotel,20 rue de la Gaité
8,75012,Mercure Paris Gare de Lyon TGV hotel,2 place Louis Armand
9,75013,Mercure Paris Bercy Bibliothèque Hotel,"6, boulevard Vincent Auriol"


In [12]:
hotels.shape

(40, 3)

In [13]:
# Use geopy library to get the latitude and longitude values of Paris city
address = 'Paris,France'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Paris are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Paris are 48.8566101, 2.3514992.


In [23]:
# Assign based on postal code the latitude and longitode coordinates
geolocator = Nominatim(user_agent="my-application")
hotels['city_coord'] = hotels['PostalCode'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))
hotels.head(5)

,PostalCode,Hotel_Name,Address,city_coord
0,75008,Mercure Paris Opera Garnier Hotel,4 rue de l 'Isly,"(48.8723467155633, 2.31229448134995)"
1,75009,Scribe Paris Opéra by Sofitel,1 rue Scribe,"(48.8774803864773, 2.33681762728008)"
2,75017,Mercure Paris St Lazare Monceau hotel,99 bis Rue de Rome,"(48.8848133, 2.3028393)"
3,75002,Hôtel Stendhal Place Vendôme Paris - MGallery ...,22 rue Danielle Casanova,"(48.8672716145714, 2.34445218423201)"
4,75018,Mercure Paris Montmartre Sacré-Coeur Hotel,3 rue Caulaincourt,"(48.8900601249192, 2.3497660353874)"


In [24]:
hotels[['Latitude', 'Longitude']] = hotels['city_coord'].apply(pd.Series)

In [25]:
Hotels = hotels.drop(columns=['city_coord'])

In [26]:
Hotels

,PostalCode,Hotel_Name,Address,Latitude,Longitude
0,75008,Mercure Paris Opera Garnier Hotel,4 rue de l 'Isly,48.872347,2.312294
1,75009,Scribe Paris Opéra by Sofitel,1 rue Scribe,48.877480,2.336818
2,75017,Mercure Paris St Lazare Monceau hotel,99 bis Rue de Rome,48.884813,2.302839
3,75002,Hôtel Stendhal Place Vendôme Paris - MGallery ...,22 rue Danielle Casanova,48.867272,2.344452
4,75018,Mercure Paris Montmartre Sacré-Coeur Hotel,3 rue Caulaincourt,48.890060,2.349766
5,75010,Hôtel L'Échiquier Opéra Paris - MGallery by So...,38 Rue De L Echiquier,48.879201,2.354391
6,75015,ibis Paris Tour Montparnasse 15th,22 Avenue du Maine,48.841418,2.295159
7,75014,Mercure Paris Gare Montparnasse Hotel,20 rue de la Gaité,48.826151,2.340915
8,75012,Mercure Paris Gare de Lyon TGV hotel,2 place Louis Armand,48.842609,2.387536
9,75013,Mercure Paris Bercy Bibliothèque Hotel,"6, boulevard Vincent Auriol",48.829296,2.362434


In [27]:
# Create a map of Paris using latitude and longitude values
map_paris = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, hotels, street in zip(Hotels['Latitude'], Hotels['Longitude'], Hotels['Hotel_Name'], Hotels['Address']):
    label = '{}, {}'.format(street, hotels)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_paris)  

map_paris

Using the latitude and longitude values it was created a map of Paris and nearby areas showing based on postal code and address the position of all the hotels collected in the database. For a better analysis it was selected including data about hotels nearby Paris with different surrounding facilities.

In [28]:
# Define Foursquare Credentials and Version
CLIENT_ID = 'IP15XLXSVGNVS2OAB01MRXZ2TLBSKVAYWAOJLY4LDBRM3XL3' # your Foursquare ID
CLIENT_SECRET = 'FBROZZGDJ03YQEKCLETRLVICLJLUILMWZ02KWOHT4LOYVNMA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IP15XLXSVGNVS2OAB01MRXZ2TLBSKVAYWAOJLY4LDBRM3XL3
CLIENT_SECRET:FBROZZGDJ03YQEKCLETRLVICLJLUILMWZ02KWOHT4LOYVNMA


## 2. Data Exploration
### 2.1. Data Understanding

In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Hotel', 
                  'Hotel Latitude', 
                  'Hotel Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
# Run the above function on each hotel and create a new dataframe called hotels_venues
LIMIT = 100
radius = 500
hotels_venues = getNearbyVenues(names=Hotels['Hotel_Name'],
                                   latitudes=Hotels['Latitude'],
                                   longitudes=Hotels['Longitude']
                                  )

Mercure Paris Opera Garnier Hotel
Scribe Paris Opéra by Sofitel
Mercure Paris St Lazare Monceau hotel
Hôtel Stendhal Place Vendôme Paris - MGallery by Sofitel
Mercure Paris Montmartre Sacré-Coeur Hotel
Hôtel L'Échiquier Opéra Paris - MGallery by Sofitel
ibis Paris Tour Montparnasse 15th
Mercure Paris Gare Montparnasse Hotel
Mercure Paris Gare de Lyon TGV hotel
Mercure Paris Bercy Bibliothèque Hotel
Novotel Paris Les Halles
Aparthotel Adagio access Paris La Villette
Novotel Suites Paris Stade de France hotel
Pullman Paris La Défense
ibis Poissy
Hôtel Mercure Paris Ouest Saint Germain
ibis budget Chambourcy Saint Germain
Hôtel Le Louis Versailles Château - MGallery by Sofitel
bis Paris Coeur d'Orly Airport
Novotel Paris Orly Rungis
ibis Styles Antony Paris Sud
ibis Massy
hotelF1 Chilly Mazarin les Champarts
ibis budget Villeneuve le Roi
bis budget Vitry sur Seine N7
hotelF1 Paris Porte de Montreuil
ibis Styles Paris Val de Fontenay
ibis budget Villemomble
ibis budget Orly Chevilly Tram 7

In [32]:
# Check the size of the resulting dataframe
print(hotels_venues.shape)
hotels_venues.head(5)

(1062, 7)


,Hotel,Hotel Latitude,Hotel Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Mercure Paris Opera Garnier Hotel,48.872347,2.312294,Le Bristol,48.871804,2.314902,Hotel
1,Mercure Paris Opera Garnier Hotel,48.872347,2.312294,Bar du Bristol,48.871796,2.315028,Hotel Bar
2,Mercure Paris Opera Garnier Hotel,48.872347,2.312294,Épicure,48.872099,2.315130,French Restaurant
3,Mercure Paris Opera Garnier Hotel,48.872347,2.312294,Le 114 Faubourg,48.871817,2.314526,French Restaurant
4,Mercure Paris Opera Garnier Hotel,48.872347,2.312294,hotel amastan,48.871745,2.312307,Resort


In [33]:
# Check how many venues were returned for each hotel
hotels_venues.groupby('Hotel').count()

,Hotel Latitude,Hotel Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Hotel,,,,,,
Aparthotel Adagio access Paris La Villette,60,60,60,60,60,60
Hôtel L'Échiquier Opéra Paris - MGallery by Sofitel,63,63,63,63,63,63
Hôtel Le Louis Versailles Château - MGallery by Sofitel,48,48,48,48,48,48
Hôtel Mercure Paris Ouest Saint Germain,4,4,4,4,4,4
Hôtel Stendhal Place Vendôme Paris - MGallery by Sofitel,100,100,100,100,100,100
Mercure Paris Bercy Bibliothèque Hotel,46,46,46,46,46,46
Mercure Paris Gare Montparnasse Hotel,27,27,27,27,27,27
Mercure Paris Gare de Lyon TGV hotel,47,47,47,47,47,47
Mercure Paris Montmartre Sacré-Coeur Hotel,77,77,77,77,77,77


This table shows how many venues were returned for each hotel. Hotels that are in the centre of Paris have the most facilities (e.g. Hôtel L'Échiquier Opéra Paris - MGallery by Sofitel, Hôtel Stendhal Place Vendôme Paris - MGallery by Sofitel, Mercure Paris Opera Garnier Hotel, Novotel Paris Les Halles, Scribe Paris Opéra by Sofitel). Compaired to the hotels located further from the center (e.g. ibis Poissy, bis budget Goussainville CDG, Hôtel Mercure Paris Ouest Saint Germain, Novotel Paris Charles-de-Gaulle Airport).

In [34]:
# Find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(hotels_venues['Venue Category'].unique())))

There are 189 uniques categories.


In [35]:
#### 2.2. Analyze each hotel
# one hot encoding
hotels_onehot = pd.get_dummies(hotels_venues[['Venue Category']], prefix="", prefix_sep="")

# add hotel column back to dataframe
hotels_onehot['Hotel'] = hotels_venues['Hotel'] 

# move hotel column to the first column
fixed_columns = [hotels_onehot.columns[-1]] + list(hotels_onehot.columns[:-1])
hotels_onehot = hotels_onehot[fixed_columns]

hotels_onehot.head(5)

,Yoga Studio,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Brasserie,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridge,Burger Joint,Bus Station,Bus Stop,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Canal,Candy Store,Caribbean Restaurant,Ch'ti Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Corsican Restaurant,Cosmetics Shop,Creperie,Cupcake Shop,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Disc Golf,Dive Bar,Doner Restaurant,Donut Shop,Electronics Store,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Franconian Restaurant,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gas Station,Gastropub,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Health Food Store,Historic Site,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Insurance Office,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Locksmith,Lounge,Martial Arts Dojo,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nightclub,Office,Organic Grocery,Other Repair Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pharmacy,Pizza Place,Platform,Plaza,Poke Place,Pool,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Resort,Restaurant,Rock Climbing Spot,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Snack Place,Soccer Field,Soup Place,Southwestern French Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stables,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Toll Plaza,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Mercure Paris Opera Garnier Hotel,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Mercure Paris Opera Garnier Hotel,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Mercure Paris Opera Garnier Hotel,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [36]:
# Examine the new dataframe size
hotels_onehot.shape

(1062, 189)

In [37]:
# Group rows by hotel and by taking the mean of the frequency of occurrence of each category
hotels_grouped = hotels_onehot.groupby('Hotel').mean().reset_index()
hotels_grouped

,Hotel,Yoga Studio,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Brasserie,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridge,Burger Joint,Bus Station,Bus Stop,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Canal,Candy Store,Caribbean Restaurant,Ch'ti Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Corsican Restaurant,Cosmetics Shop,Creperie,Cupcake Shop,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Disc Golf,Dive Bar,Doner Restaurant,Donut Shop,Electronics Store,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Franconian Restaurant,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gas Station,Gastropub,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Health Food Store,Historic Site,Hostel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Insurance Office,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Locksmith,Lounge,Martial Arts Dojo,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nightclub,Office,Organic Grocery,Other Repair Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pharmacy,Pizza Place,Platform,Plaza,Poke Place,Pool,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Resort,Restaurant,Rock Climbing Spot,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Snack Place,Soccer Field,Soup Place,Southwestern French Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stables,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Toll Plaza,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Aparthotel Adagio access Paris La Villette,0.000000,0.000000,0.016667,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016667,0.150000,0.000000,0.033333,0.000000,0.016667,0.000000,0.000000,0.016667,0.000000,0.00,0.000000,0.016667,0.00,0.00,0.000000,0.016667,0.00,0.016667,0.000000,0.000000,0.033333,0.00,0.000000,0.016667,0.00,0.000000,0.00,0.016667,0.000000,0.00,0.00,0.000000,0.016667,0.016667,0.000000,0.016667,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.016667,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.016667,0.000000,0.00,0.0,0.016667,0.0,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.00,0.016667,0.000000,0.000000,0.000000,0.000000,0.00,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.016667,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.016667,0.000000,0.016667,0.000000,0.000000,0.016667,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.0,0.000000,0.0000

This table shows what venues are available near each hotel. There is a huge variety of facilities from local and foreign restaurants (french, african, american, argentinian, asian, belgian, chinese, hawaiian and so on), airport services, art gallery, art museum, bus and trail station, shops, church, comedy club, grocery stores, market to jazz club, medical center, museum, nightclub, park, theater and many others.

In [38]:
# Confirm the new size
hotels_grouped.shape

(34, 189)

In [39]:
# Print each hotel along with the top 5 most common venues
num_top_venues = 5

for Code in hotels_grouped['Hotel']:
    print("Hotel:", Code)
    temp = hotels_grouped[hotels_grouped['Hotel'] == Code].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

Hotel: Aparthotel Adagio access Paris La Villette
                venue  freq
0                 Bar  0.15
1   French Restaurant  0.08
2          Restaurant  0.05
3    Sushi Restaurant  0.03
4  Italian Restaurant  0.03


Hotel: Hôtel L'Échiquier Opéra Paris - MGallery by Sofitel
               venue  freq
0  French Restaurant  0.16
1         Restaurant  0.06
2  Indian Restaurant  0.05
3        Coffee Shop  0.05
4             Bakery  0.05


Hotel: Hôtel Le Louis Versailles Château - MGallery by Sofitel
               venue  freq
0  French Restaurant  0.21
1                Pub  0.08
2              Plaza  0.06
3           Creperie  0.06
4         Restaurant  0.04


Hotel: Hôtel Mercure Paris Ouest Saint Germain
                  venue  freq
0             Wine Shop  0.25
1          Soccer Field  0.25
2         Train Station  0.25
3  Gym / Fitness Center  0.25
4           Yoga Studio  0.00


Hotel: Hôtel Stendhal Place Vendôme Paris - MGallery by Sofitel
               venue  freq
0  French 

We can observe for each hotel the top 5 most common venues. Hotels from central area have in particular facilities, such as: local and foreign restaurants, bistro, cafe, bar, bakery, stores, plaza, lounge, along with the biggest tourist attractions: The Eiffel Tower, Louvre Museum, Notre Dame de Paris, Pantheon, Arc de Triomphe.

In [40]:
#  Put that into a pandas dataframe and sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
# Create the new dataframe and display the top 10 venues for each hotel
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Hotel']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [42]:
# Show for each hotel the most common venue
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Hotel'] = hotels_grouped['Hotel']

for ind in np.arange(hotels_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(hotels_grouped.iloc[ind, :], num_top_venues)

venues_sorted.head()

,Hotel,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aparthotel Adagio access Paris La Villette,Bar,French Restaurant,Restaurant,Beer Bar,Café,Supermarket,Sushi Restaurant,Seafood Restaurant,Italian Restaurant,Japanese Restaurant
1,Hôtel L'Échiquier Opéra Paris - MGallery by So...,French Restaurant,Restaurant,Indian Restaurant,Coffee Shop,Bakery,Hostel,Japanese Restaurant,Bistro,Bar,Department Store
2,Hôtel Le Louis Versailles Château - MGallery b...,French Restaurant,Pub,Creperie,Plaza,Bistro,Pizza Place,Restaurant,Bakery,Steakhouse,Juice Bar
3,Hôtel Mercure Paris Ouest Saint Germain,Wine Shop,Soccer Field,Gym / Fitness Center,Train Station,Diner,Farmers Market,Falafel Restaurant,Exhibit,Electronics Store,Donut Shop
4,Hôtel Stendhal Place Vendôme Paris - MGallery ...,French Restaurant,Wine Bar,Cocktail Bar,Bakery,Bistro,Italian Restaurant,Salad Place,Thai Restaurant,Burger Joint,Boutique


### 3. Data Modeling

In [43]:
# Run k-means to cluster the neighborhood into 7 clusters
# Set number of clusters
kclusters = 7
hotels_grouped_clustering = hotels_grouped.drop('Hotel', 1)

# Run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hotels_grouped_clustering)

# Check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 5, 5, 5, 5, 5, 0, 4, 3, 5, 2,
       5, 5, 5, 6, 5, 1, 1, 1, 5, 6, 5], dtype=int32)

In [44]:
# Add labels
venues_sorted.insert(0,'Cluster Labels', kmeans.labels_)
hotels_merged = Hotels
hotels_merged.rename(columns={'Hotel_Name': 'Hotel'}, inplace=True)

In [45]:
# merge toronto_grouped with toronto to add latitude/longitude for each neighborhood
hotels_merged = hotels_merged.join(venues_sorted.set_index('Hotel'), on='Hotel')
hotels_merged.head()

,PostalCode,Hotel,Address,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,75008,Mercure Paris Opera Garnier Hotel,4 rue de l 'Isly,48.872347,2.312294,5.0,French Restaurant,Italian Restaurant,Bakery,Art Gallery,Japanese Restaurant,Salad Place,Spa,Bar,Cocktail Bar,Clothing Store
1,75009,Scribe Paris Opéra by Sofitel,1 rue Scribe,48.877480,2.336818,5.0,French Restaurant,Bistro,Cocktail Bar,Bar,Japanese Restaurant,Theater,Italian Restaurant,Lounge,Café,Bakery
2,75017,Mercure Paris St Lazare Monceau hotel,99 bis Rue de Rome,48.884813,2.302839,5.0,French Restaurant,Italian Restaurant,Bakery,Bistro,Plaza,Asian Restaurant,Bar,Café,Restaurant,Supermarket
3,75002,Hôtel Stendhal Place Vendôme Paris - MGallery ...,22 rue Danielle Casanova,48.867272,2.344452,5.0,French Restaurant,Wine Bar,Cocktail Bar,Bakery,Bistro,Italian Restaurant,Salad Place,Thai Restaurant,Burger Joint,Boutique
4,75018,Mercure Paris Montmartre Sacré-Coeur Hotel,3 rue Caulaincourt,48.890060,2.349766,5.0,French Restaurant,Bar,Coffee Shop,Café,Pizza Place,Bistro,Convenience Store,Middle Eastern Restaurant,Plaza,Fast Food Restaurant


In [46]:
hotels_merged = hotels_merged.fillna(0)
hotels_merged['Cluster Labels']=hotels_merged['Cluster Labels'].astype(int)

In [47]:
# Visualize the resulting clusters
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(hotels_merged['Latitude'], hotels_merged['Longitude'], hotels_merged['Hotel'], hotels_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The map shows the cluster grouping mode. It can be noticed that hotels with common venues and many tourist attractions are grouped together, compared with hotels located in less crowded areas and with fewer facilities and venues.

In [48]:
#  Examine each cluster
# 1st cluster
hotels_merged.loc[hotels_merged['Cluster Labels'] == 0, hotels_merged.columns[[1] + list(range(5, hotels_merged.shape[1]))]]

,Hotel,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Novotel Suites Paris Stade de France hotel,0,0,0,0,0,0,0,0,0,0,0
18,bis Paris Coeur d'Orly Airport,0,German Restaurant,Franconian Restaurant,Restaurant,Women's Store,Diner,Falafel Restaurant,Exhibit,Electronics Store,Donut Shop,Doner Restaurant
19,Novotel Paris Orly Rungis,0,0,0,0,0,0,0,0,0,0,0
21,ibis Massy,0,0,0,0,0,0,0,0,0,0,0
32,Hotel Mercure Paris CDG Airport & Convention,0,0,0,0,0,0,0,0,0,0,0
34,ibis budget Roissy CDG Paris Nord 2,0,0,0,0,0,0,0,0,0,0,0
36,ibis Villepinte Parc Expos,0,0,0,0,0,0,0,0,0,0,0


In [49]:
# 2nd cluster
hotels_merged.loc[hotels_merged['Cluster Labels'] == 1, hotels_merged.columns[[1] + list(range(5, hotels_merged.shape[1]))]]

,Hotel,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,ibis budget Chambourcy Saint Germain,1,American Restaurant,Deli / Bodega,Medical Center,Café,Women's Store,Dive Bar,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Exhibit
26,ibis Styles Paris Val de Fontenay,1,Pizza Place,Bowling Alley,Pub,Restaurant,Park,Dessert Shop,Falafel Restaurant,Exhibit,Electronics Store,Donut Shop
35,ibis Styles Parc des Expositions de Villepinte,1,American Restaurant,Café,Pizza Place,Sports Bar,Grocery Store,Women's Store,Disc Golf,Farmers Market,Falafel Restaurant,Exhibit


In [50]:
# 3rd cluster
hotels_merged.loc[hotels_merged['Cluster Labels'] == 2, hotels_merged.columns[[1] + list(range(5, hotels_merged.shape[1]))]]

,Hotel,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
39,hotelF1 Aulnay Garonor A1,2,Flower Shop,Women's Store,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Exhibit,Electronics Store,Donut Shop,Doner Restaurant


In [51]:
# 4th cluster
hotels_merged.loc[hotels_merged['Cluster Labels'] == 3, hotels_merged.columns[[1] + list(range(5, hotels_merged.shape[1]))]]

,Hotel,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,bis budget Goussainville CDG,3,Construction & Landscaping,Women's Store,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Exhibit,Electronics Store,Donut Shop,Doner Restaurant


In [52]:
# 5th cluster
hotels_merged.loc[hotels_merged['Cluster Labels'] == 4, hotels_merged.columns[[1] + list(range(5, hotels_merged.shape[1]))]]

,Hotel,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,bis Styles Paris Charles de Gaulle Airport,4,Rock Climbing Spot,Women's Store,Dessert Shop,Farmers Market,Falafel Restaurant,Exhibit,Electronics Store,Donut Shop,Doner Restaurant,Dive Bar


In [53]:
# 6th cluster
hotels_merged.loc[hotels_merged['Cluster Labels'] == 5, hotels_merged.columns[[1] + list(range(5, hotels_merged.shape[1]))]]

,Hotel,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Mercure Paris Opera Garnier Hotel,5,French Restaurant,Italian Restaurant,Bakery,Art Gallery,Japanese Restaurant,Salad Place,Spa,Bar,Cocktail Bar,Clothing Store
1,Scribe Paris Opéra by Sofitel,5,French Restaurant,Bistro,Cocktail Bar,Bar,Japanese Restaurant,Theater,Italian Restaurant,Lounge,Café,Bakery
2,Mercure Paris St Lazare Monceau hotel,5,French Restaurant,Italian Restaurant,Bakery,Bistro,Plaza,Asian Restaurant,Bar,Café,Restaurant,Supermarket
3,Hôtel Stendhal Place Vendôme Paris - MGallery ...,5,French Restaurant,Wine Bar,Cocktail Bar,Bakery,Bistro,Italian Restaurant,Salad Place,Thai Restaurant,Burger Joint,Boutique
4,Mercure Paris Montmartre Sacré-Coeur Hotel,5,French Restaurant,Bar,Coffee Shop,Café,Pizza Place,Bistro,Convenience Store,Middle Eastern Restaurant,Plaza,Fast Food Restaurant
5,Hôtel L'Échiquier Opéra Paris - MGallery by So...,5,French Restaurant,Restaurant,Indian Restaurant,Coffee Shop,Bakery,Hostel,Japanese Restaurant,Bistro,Bar,Department Store
6,ibis Paris Tour Montparnasse 15th,5,French Restaurant,Italian Restaurant,Bistro,Bakery,Lebanese Restaurant,Coffee Shop,Restaurant,Brasserie,Indian Restaurant,Park
7,Mercure Paris Gare Montparnasse Hotel,5,Gym,Vietnamese Restaurant,Bus Stop,Café,Indian Restaurant,Park,Restaurant,Cheese Shop,Pharmacy,Peruvian Restaurant
8,Mercure Paris Gare de Lyon TGV hotel,5,Supermarket,Bistro,Chinese Restaurant,Indian Restaurant,Bus Station,Japanese Restaurant,Soup Place,Sandwich Place,Electronics Store,Lebanese Restaurant
9,Mercure Paris Bercy Bibliothèque Hotel,5,Vietnamese Restaurant,Asian Restaurant,Chinese Restaurant,Thai Restaurant,French Restaurant,Plaza,Sushi Restaurant,Park,Pizza Place,Paper / Office Supplies Store


In [54]:
# 7th cluster
hotels_merged.loc[hotels_merged['Cluster Labels'] == 6, hotels_merged.columns[[1] + list(range(5, hotels_merged.shape[1]))]]

,Hotel,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,ibis Poissy,6,Gas Station,Sandwich Place,Women's Store,Diner,Farmers Market,Falafel Restaurant,Exhibit,Electronics Store,Donut Shop,Doner Restaurant
27,ibis budget Villemomble,6,Plaza,Restaurant,Gas Station,Dessert Shop,Falafel Restaurant,Exhibit,Electronics Store,Donut Shop,Doner Restaurant,Dive Bar
30,Novotel Paris Charles-de-Gaulle Airport,6,Thai Restaurant,Gas Station,Asian Restaurant,Dessert Shop,Farmers Market,Falafel Restaurant,Exhibit,Electronics Store,Donut Shop,Doner Restaurant


## IV. Result Section
Using the latitude and longitude values it was created a map of Paris and nearby areas showing based on postal code and address the position of all the hotels collected in the database. For a better analysis it was selected including data about hotels nearby Paris with different surrounding facilities.

It was observed what venues are available near each hotel. Hotels that are in the center of Paris have the most facilities (e.g. Hôtel L'Échiquier Opéra Paris - MGallery by Sofitel, Hôtel Stendhal Place Vendôme Paris - MGallery by Sofitel, Mercure Paris Opera Garnier Hotel, Novotel Paris Les Halles, Scribe Paris Opéra by Sofitel). Compared to the hotels located further from the center (e.g. ibis Poissy, bis budget Goussainville CDG, Hôtel Mercure Paris Ouest Saint Germain, Novotel Paris Charles-de-Gaulle Airport). There is a huge variety of facilities from local and foreign restaurants (french, african, american, argentinian, asian, belgian, chinese, hawaiian and so on), airport services, art gallery, art museum, bus and trail station, shops, church, comedy club, grocery stores, market to jazz club, medical center, museum, nightclub, park, theater and many others. Hotels from central area have in particular facilities, such as: local and foreign restaurants, bistro, cafe, bar, bakery, stores, plaza, lounge, along with the biggest tourist attractions: The Eiffel Tower, Louvre Museum, Notre Dame de Paris, Pantheon, Arc de Triomphe.

The map and the last 7 tables show the cluster grouping mode. The clusters were grouped by similarities. It can be noticed that hotels with common venues and many tourism attractions are grouped together, compared with hotels located in less crowded areas and with fewer facilities and venues. The largest cluster, the last one, contains the most number of hotels. They have the largest number of facilities and benefit of many tourist attractions. These are also the most expensive ones. The price of accommodation differs depending on the facilities and surroundings. The closer the hotel is to the tourist attractions, the more expensive it will be.

## V. Discussion Section
You will find a huge variety of hotel choices when you are planning to visit France. The cost of accommodation varies according to region and type. While hotel rooms are inevitably more expensive in Paris, there are also many interesting hotels nearby Paris.

It is interesting to note that, although the hotels in the center of the Paris (e.g. Hôtel L'Échiquier Opéra Paris - MGallery by Sofitel, Hôtel Stendhal Place Vendôme Paris - MGallery by Sofitel, Mercure Paris Opera Garnier Hotel, Novotel Paris Les Halles, Scribe Paris Opéra by Sofitel) might be considered very expensive due to all the surrounding facilities (such as local and foreign restaurants (french, african, american, argentinian, asian, belgian, chinese, hawaiian and so on), airport services, art gallery, art museum, bus and trail station, shops, church, comedy club, grocery stores, market, medical center, museum, nightclub, park, theater), hotels nearby the center have also a wide range of facilities and tourist attractions: Parc des Expositions, Rolland Garros, Palace of Versailles, Stade de France, Parc de la Villette and so on.

Although, all the clusters have an optimal range of facilities, I have found two main patterns. The first pattern I am referring to, clusters 0, 1, 2, 3, 4, and 5, highlights the hotels that have a smaller number of facilities and are located in less crowded areas. Instead, the second pattern, cluster 6, highlights the hotels that have the largest number of facilities and benefit of many tourist attractions.

## VI. Conclusion Section
To solve this business problem, we clustered hotels from Paris and nearby areas in order to recommend venues and to support people who are looking for the right hotel to take the best decisions.

We gathered data from https://www.accorhotels.com and Forsquare. We explore and target recommended locations across different venues, we extract, analyze and discover hidden insigths in data and finally we used the k-means clustering technique to cluster Paris and nearby hotels to see how they group togheted with the final puurpose to provide useful information related to accomodation strategies.

In conclusion, Paris is one of the most popular destinations in Europe and not only. It offers a wide range of tourist attractions: food, fashion, culture, nature, art and so on. If someone wants to travel there are many things to consider from choosing the right location, accommodation, flights, rental cars to attractions, restaurants, stores and other facilities. The price of accommodation depends on the number of venues and surrounding facilities. The closer the hotel is to the tourist attractions, the more expensive it will be. However the analyzes show that there are very good hotels at a very short distance from the center and that it is not necessarily mandatory to stay in the city center to visit the most beautiful attractions.